# NEW TEST 

In [4]:
import pandas as pd
import numpy as np
import time

from sklearn.manifold import MDS
from sklearn.cluster import AgglomerativeClustering
from sqlalchemy import create_engine
from datetime import datetime
from dateutil.relativedelta import relativedelta

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

engine = create_engine('sqlite:///fund.db')
output_notebook()

Loading BokehJS ...

In [5]:
start = "2015-01"
end = "2015-12"
investement_type = "不分類"
sharpe_ratio = 0.4
std = 0.3
beta = 0.2
treynor_ratio = 0.3
money = 10000
buy_ratio = np.asarray((0.3,0.4,0.2,0.1), dtype=np.float)
strategy = 1
frequency = 3
profit = pd.DataFrame()
hold = np.zeros([4], dtype=np.float)
choose = np.asarray([" ", " ", " ", " "], dtype='<U32')

In [6]:
start_unix = time.mktime((datetime.strptime(start, '%Y-%m') - relativedelta(years=+1)).timetuple())
end_unix = time.mktime((datetime.strptime(start, '%Y-%m') - relativedelta(days=+1)).timetuple())
data_df = pd.read_sql(sql='select * from price where date between ? and ? order by date asc',
               con=engine, params=[start_unix, end_unix])
data_df = data_df.pivot(index='date', columns='fund_id', values='nav')
data_df = data_df.fillna(method="ffill")
data_df = data_df.fillna(method="bfill")

In [8]:
indicator_Rp = (data_df - data_df.iloc[0]) / data_df.iloc[0]
indicator_σp = indicator_Rp.std(ddof=1)
indicator_ρpm = indicator_Rp.corr()["0050 元大台灣50"]
indicator_σm = indicator_σp["0050 元大台灣50"]
indicator_βp = indicator_ρpm * indicator_σp / indicator_σm
bl = data_df.iloc[0] > 0

In [9]:
bl = bl & ((indicator_Rp.iloc[-1] - 0.01) / indicator_σp > sharpe_ratio)
bl = bl & (data_df.std(ddof=1) < std)
bl = bl & (indicator_βp < beta)
bl = bl & ((indicator_Rp.iloc[-1] - 0.01) / indicator_βp > treynor_ratio)
data_df = data_df.T[bl].T

In [10]:
data_df = data_df.pct_change()

In [11]:
data_df = data_df.corr()

In [12]:
camp = pd.DataFrame(AgglomerativeClustering(n_clusters=4).fit(data_df).labels_,index=data_df.index,columns=['label'])

In [13]:
for i,ch in enumerate(choose):
    if ch in camp.index:
        camp = camp.drop(camp[camp.label == camp.loc[ch].label].index ,axis=0)
    else:
        temp = camp.sample(n=1)
        camp = camp.drop(camp[camp.label == temp.label[0]].index, axis=0)
        choose[i] = temp.index[0]

In [14]:
a = pd.DataFrame()

In [15]:
i = 0
start_unix = time.mktime((datetime.strptime(start, '%Y-%m') + relativedelta(months=i)).timetuple())
end_unix = time.mktime((datetime.strptime(start, '%Y-%m') + relativedelta(months=i+1, days=-1)).timetuple())
data_df = pd.read_sql(sql='select * from price where date between ? and ? order by date asc',
               con=engine, params=[start_unix, end_unix])
data_df = data_df.pivot(index='date', columns='fund_id', values='nav')
data_df = data_df.fillna(method="ffill")
data_df = data_df.fillna(method="bfill")

In [23]:
data_df[choose].iloc[-1].mean()

10.4525

In [12]:
if strategy == 0:
    hold += (buy_ratio * money / data_df[choose].iloc[0].T).values
    profit = pd.concat(
        [profit, (data_df[choose] * hold).T.sum() / (money*(i+1))], axis=0)
else:
    if i == 0:
        hold = (buy_ratio * money / data_df[choose].iloc[0].T).values
    elif strategy != 1 and i % frequency == frequency - 1:
        temp = (hold * data_df[choose].iloc[0]).sum()
        if strategy == 3:
            choose = selection(start, investement_type, i, sharpe_ratio,std, beta, treynor_ratio, choose)
        hold = (buy_ratio * temp /
                data_df[choose].iloc[0].T).values
    profit = pd.concat(
        [profit, (data_df[choose] * hold).T.sum() / money], axis=0)

In [13]:
choose[3] = "0050 元大台灣50"

In [14]:
a = pd.concat([a,data_df[choose]], sort=False)

In [15]:
for j,ch in enumerate(choose):
    interest = pd.read_sql(sql='select sum(interest) from interest where date between ? and ? and fund_id == ? order by date asc',
                con=engine, params=[start_unix, end_unix, ch])
    hold[j] +=(interest * hold[j] / data_df[ch].iloc[-1]).fillna(0).loc[0][0]

In [16]:
a.std().mean()

0.5409105642482758

In [17]:
data_df = pd.concat([data_df[choose], data_df.T.sample(n=296).T], axis=1).T.drop_duplicates().T

In [18]:
data_df = data_df.pct_change()

In [19]:
data_df = data_df.corr()

In [20]:
data_df = 1 - data_df * 0.5 - 0.5

In [21]:
data_df = data_df.fillna(-1)

In [29]:
data_df[choose].T[choose].sum().sum()/12

0.5393197417968485

In [75]:
color = np.asarray(["yellow" for i in range(len(data_df))])
color[0:4] = "purple"

In [76]:
mds = MDS(n_components=2, dissimilarity='precomputed', n_jobs=-1).fit(data_df).embedding_
source = ColumnDataSource(data=dict(x=mds[:, 0],
                                    y=mds[:, 1],
                                    name=data_df.index,
                                    color=color,
))
TOOLTIPS = [
    ("fund_id", "@name"),
]
p = figure(plot_width=300, plot_height=300, tooltips=TOOLTIPS,
           title="MDS", toolbar_location=None, tools="")
p.circle(x='x', y='y', color='color', size=6.5, source=source)
show(p)

In [78]:
profit = profit.rename(columns={0:"profit"})
profit["profit"] = profit["profit"]-1
profit.index.name = "date"
profit.index = profit.index + 28800
profit.index = pd.to_datetime(profit.index, unit='s')

In [49]:
p = figure(x_axis_type="datetime", plot_width=900,
           plot_height=300, title="Profit", toolbar_location=None, tools="")
p.line(x='date', y='profit', line_width=3, source=profit)
p.add_tools(HoverTool(tooltips=[("date", "@date{%F}"), ("profit", "@profit%")],
                      formatters={'date': 'datetime', }, mode='vline'))
show(p)

In [85]:
(hold * data_df.iloc[-1][choose])

fund_id
ISZK8 景順全歐洲AUSDh$    0.0
T0595A 元大人民幣貨幣市      0.0
T2289Y 復華全球戰略配置      0.0
0050 元大台灣50          0.0
Name: GSF83 天達歐高收債C2AH, dtype: float64

In [70]:
a = pd.DataFrame([[1,2,3]])
b = pd.DataFrame([[1,2],[1,2]])
pd.concat([a, b], axis=1, join="inner")

,0,1,2,0,1
0,1,2,3,1,2


In [71]:
pd.DataFrame([[1,2],[1,2]])

,0,1
0,1,2
1,1,2


In [87]:
buy_ratio = np.asarray([float(1 + i) for i in range(4)], dtype=np.float)

In [88]:
buy_ratio

array([1., 2., 3., 4.])

In [17]:
from collections import OrderedDict, defaultdict

In [3]:
df = pd.read_sql(sql='select * from basic_information limit 10', con=engine)

In [30]:
dd = defaultdict(list)
df.to_dict('records', into=dd)

[defaultdict(list,
             {'fund_id': '0050 元大台灣50',
              'chinese_name': '元大台灣卓越50證券投資信託基金',
              'english_name': 'Yuanta/P-shares Taiwan Top 50 ETF',
              'isin_code': 'TW0000050004',
              'entry_day': '2003-06-25',
              'manager_fee': 0.32,
              'custody_fee': 0.035,
              'sales_fee': 2.0,
              'area': '境內'}),
 defaultdict(list,
             {'fund_id': '0051 元大中型100',
              'chinese_name': '元大台灣中型 100證券投資信託基金',
              'english_name': 'Yuanta/P-shares Taiwan Mid-Cap 100 ETF',
              'isin_code': 'TW0000051002',
              'entry_day': '2006-08-24',
              'manager_fee': 0.4,
              'custody_fee': 0.035,
              'sales_fee': 2.0,
              'area': '境內'}),
 defaultdict(list,
             {'fund_id': '0052 富邦科技',
              'chinese_name': '富邦台灣科技指數證券投資信託基金',
              'english_name': 'FUBON TAIWAN TECHNOLOGY TRACKER FUND',
              'isin_code': 'TW